In [2]:
pip install linearmodels

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyfixest

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install adjustText

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install matplotlib seaborn

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7571
    CPU family:           23
    Model:                1
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             2
    BogoMIPS:             4399.87
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid aperfmperf ts
                          c_known_freq pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_
                          2 movbe popcnt aes xsave avx f16c rdrand hypervisor la


In [7]:
import pandas as pd
import numpy as np
import snowflake.connector as snow
import statsmodels.api as sm
import gc
import warnings
import os
from linearmodels import PanelOLS, PooledOLS
import pickle
import config
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from pyfixest import feols
import seaborn as sns
import textwrap

In [8]:
# Set up your connection parameters
user = config.credentials['USERNAME']
password = config.credentials['PASSWORD']
account = 'PCA67849'
warehouse = config.credentials['WAREHOUSE']

# Connect to Snowflake
conn = snow.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database='',
    schema=''
)

INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.15.0, Python Version: 3.10.12, Platform: Linux-6.8.0-1033-aws-x86_64-with-glibc2.35
INFO:snowflake.connector.connection:Connecting to GLOBAL Snowflake domain


In [9]:
# Start query with half a million sample
combined_query = f"""
WITH base_postings AS (
    -- Get base postings data with sampling (500,000 per SOC per year)
    SELECT 
        ID,
        POSTED,
        CASE 
            WHEN SALARY IS NOT NULL THEN SALARY
            ELSE (SALARY_TO + SALARY_FROM) / 2 
        END as salary_imputed,
        ONET_2019,
        TITLE_CLEAN,
        SOC_2021_2,
        SOC_2021_2_NAME,
        SOC_2021_5,           -- Added SOC_2021_5 for fixed effects
        SOC_2021_5_NAME,      -- Added SOC_2021_5_NAME for readability
        COALESCE(MIN_EDULEVELS_NAME, 'Missing') as MIN_EDULEVELS_NAME,
        COALESCE(MIN_YEARS_EXPERIENCE, 0) as MIN_YEARS_EXPERIENCE,
        YEAR(POSTED) as YEAR
    FROM BGI_POSTINGS_BACKUPS.FEB_25.US_POSTINGS
    WHERE YEAR(POSTED) BETWEEN 2022 AND 2024
        AND ONET_2019 IS NOT NULL
        AND (SALARY_TO > 0 OR SALARY > 0 OR SALARY_FROM > 0)
        AND ARRAY_CONTAINS('Company'::VARIANT, SOURCE_TYPES)
        AND IS_INTERNSHIP = FALSE
        AND COMPANY_IS_STAFFING = FALSE
        AND SOC_2021_2 IN ('11-0000', '13-0000', '15-0000', '17-0000', '19-0000') -- white collar
        -- AND SOC_2021_2 IN ('47-0000', '49-0000', '51-0000') -- blue collar
        -- Sample 500,000 per SOC per year
        -- QUALIFY ROW_NUMBER() OVER (PARTITION BY YEAR(POSTED), SOC_2021_2 ORDER BY RANDOM()) <= 500000
),
skill_counts AS (
    -- Calculate skill counts (removed minimum threshold)
    SELECT 
        SKILL_NAME,
        COUNT(DISTINCT ps.ID) as skill_count
    FROM BGI_POSTINGS_BACKUPS.FEB_25.US_POSTINGS_SKILLS ps
    INNER JOIN base_postings bp ON ps.ID = bp.ID
    GROUP BY SKILL_NAME
    HAVING COUNT(DISTINCT ps.ID) >= 500
)

-- Final query
SELECT 
    bp.*,
    ps.SKILL_NAME,
    sc.skill_count
FROM base_postings bp
LEFT JOIN BGI_POSTINGS_BACKUPS.FEB_25.US_POSTINGS_SKILLS ps ON bp.ID = ps.ID
LEFT JOIN skill_counts sc ON ps.SKILL_NAME = sc.SKILL_NAME
"""
# Execute query and get data
reg_df = pd.read_sql(combined_query, conn)

In [10]:
# First get the skills list
skills_query = """
SELECT "Skill" 
FROM TEMPORARY_DATA.JNANIA."ADE_LC_skills_v5_2025_03_10"
"""
skills = pd.read_sql(skills_query, conn)['Skill'].tolist()

In [11]:
# Calculate skill statistics
skill_stats = reg_df[reg_df['SKILL_NAME'].isin(skills)].groupby('SKILL_NAME').agg({
    'SOC_2021_2': 'nunique',  # Changed from ONET_2019 to SOC_2021_2
    'MIN_EDULEVELS_NAME': 'nunique',
    'ID': 'count'
}).rename(columns={
    'SOC_2021_2': 'soc_count',  # Renamed from occ_count to soc_count
    'MIN_EDULEVELS_NAME': 'ed_count',
    'ID': 'obs_count'
})

# Filter valid skills
print("Filtering valid skills...")
include_skills = skill_stats[
    (skill_stats['soc_count'] > 1) & 
    (skill_stats['ed_count'] > 1)  # Fixed: removed 'of'
].index.tolist()
print(f"Number of skills to process: {len(include_skills)}")

Filtering valid skills...
Number of skills to process: 101


In [12]:
# Query to get share of jobs that have ade skills
skill_occ_query = f"""
WITH base_postings AS (
    SELECT COUNT(DISTINCT ID) AS total_postings
    FROM BGI_POSTINGS_BACKUPS.FEB_25.US_POSTINGS
    WHERE YEAR(POSTED) BETWEEN 2022 AND 2024 --- pooling 3 years of data
        AND SOC_2021_2 IS NOT NULL
        AND SOC_2021_2 != '99-0000'
        AND ARRAY_SIZE(SKILLS) > 0
        AND (SALARY_TO > 0 OR SALARY > 0 OR SALARY_FROM > 0)
        AND ARRAY_CONTAINS('Company'::VARIANT, SOURCE_TYPES)
        AND IS_INTERNSHIP = FALSE
        AND COMPANY_IS_STAFFING = FALSE
),
postings_with_ade_skills AS (
    SELECT COUNT(DISTINCT bp.ID) AS skill_postings
    FROM BGI_POSTINGS_BACKUPS.FEB_25.US_POSTINGS bp
    JOIN BGI_POSTINGS_BACKUPS.FEB_25.US_POSTINGS_SKILLS ps ON bp.ID = ps.ID
    WHERE YEAR(bp.POSTED) BETWEEN 2022 AND 2024
        AND bp.SOC_2021_2 IS NOT NULL
        AND bp.SOC_2021_2 != '99-0000'
        AND ARRAY_SIZE(bp.SKILLS) > 0
        AND (bp.SALARY_TO > 0 OR bp.SALARY > 0 OR bp.SALARY_FROM > 0)
        AND ARRAY_CONTAINS('Company'::VARIANT, bp.SOURCE_TYPES)
        AND bp.IS_INTERNSHIP = FALSE
        AND bp.COMPANY_IS_STAFFING = FALSE
        AND ps.SKILL_NAME IN ({', '.join([f"'{skill}'" for skill in skills])})
)
SELECT 
    bp.total_postings,
    pws.skill_postings,
    ROUND((pws.skill_postings * 100.0 / bp.total_postings), 2) AS ade_skill_share_pct
FROM base_postings bp, postings_with_ade_skills pws;
"""

# Execute query and get data
skills_share = pd.read_sql(skill_occ_query, conn)

In [13]:
skills_share

,TOTAL_POSTINGS,SKILL_POSTINGS,ADE_SKILL_SHARE_PCT
0,6801838,2793188,41.07


In [14]:
# # Get counts by SOC-5 level for each skill
# soc5_counts = reg_df.groupby(['SKILL_NAME', 'SOC_2021_5', 'SOC_2021_5_NAME', 'SOC_2021_2', 'SOC_2021_2_NAME']).size().reset_index(name='count')

# # Sort by count within each skill
# soc5_counts_sorted = soc5_counts.sort_values(['SKILL_NAME', 'count'], ascending=[True, False])

# # Get top 6 SOC-5 categories for each skill
# top_soc5 = soc5_counts_sorted.groupby('SKILL_NAME').head(6)

# # Add a percentage column to show distribution
# top_soc5['percentage'] = top_soc5.groupby('SKILL_NAME')['count'].transform(lambda x: x / x.sum() * 100)

# # Print the results
# print("Top 6 SOC-5 categories for each skill:")
# print(top_soc5[['SKILL_NAME', 'SOC_2021_5', 'SOC_2021_5_NAME', 'SOC_2021_2_NAME', 'count', 'percentage']])

# # Also summarize by SOC-2 level
# soc2_counts = reg_df.groupby(['SKILL_NAME', 'SOC_2021_2', 'SOC_2021_2_NAME']).size().reset_index(name='count')
# soc2_counts['percentage'] = soc2_counts.groupby('SKILL_NAME')['count'].transform(lambda x: x / x.sum() * 100)
# soc2_counts_sorted = soc2_counts.sort_values(['SKILL_NAME', 'count'], ascending=[True, False])

# print("\nDistribution by SOC-2 categories:")
# print(soc2_counts_sorted[['SKILL_NAME', 'SOC_2021_2', 'SOC_2021_2_NAME', 'count', 'percentage']])

In [15]:
def create_regression_dataset(skill, full_data): 
    """
    Creates regression dataset for a given skill with proper control group using SOC_2021_5 fixed effects
    
    Parameters:
    skill (str): The skill to analyze
    full_data (DataFrame): The complete dataset with job postings
    
    Returns:
    tuple: (regression_df, skill_df, nonskill_df) - The prepared datasets for regression
    """
    try:
        # Define needed columns to reduce memory usage
        needed_columns = ['ID', 'SALARY_IMPUTED', 'YEAR', 
                          'SOC_2021_5', 'SOC_2021_5_NAME',
                          'SOC_2021_2', 'SOC_2021_2_NAME', 
                          'MIN_EDULEVELS_NAME', 'MIN_YEARS_EXPERIENCE', 'SKILL_NAME']
        
        # Create subset of posts that have the skill (treatment group)
        skill_filter = full_data['SKILL_NAME'] == skill
        skill_df = full_data.loc[skill_filter, needed_columns].copy()
        skill_posts_ids = skill_df['ID'].unique()  # Get unique IDs
        
        # Remove duplicates - this is important for accurate counting
        skill_df = skill_df.drop_duplicates(
            subset=['ID', 'SALARY_IMPUTED', 'YEAR',
                    'SOC_2021_5', 'SOC_2021_2', 
                    'MIN_EDULEVELS_NAME', 'MIN_YEARS_EXPERIENCE']
        )
        skill_df['has_skill'] = 1
        
        # Treatment group count = len(skill_df)
        
        # Process control group in chunks
        chunk_size = 10000  
        nonskill_chunks = []
        
        for chunk_start in range(0, len(full_data), chunk_size):
            chunk_end = min(chunk_start + chunk_size, len(full_data))
            chunk = full_data.iloc[chunk_start:chunk_end]
            
            # Filter for control observations (must NOT have the current skill)
            nonskill_chunk = chunk[~chunk['ID'].isin(skill_posts_ids)]
            if len(nonskill_chunk) > 0:
                nonskill_chunk = nonskill_chunk[needed_columns].copy()
                # Remove duplicates for accurate counting
                nonskill_chunk = nonskill_chunk.drop_duplicates(
                    subset=['ID', 'SALARY_IMPUTED', 'YEAR',
                            'SOC_2021_5', 'SOC_2021_2',
                            'MIN_EDULEVELS_NAME', 'MIN_YEARS_EXPERIENCE']
                )
                nonskill_chunk['has_skill'] = 0
                nonskill_chunks.append(nonskill_chunk)
            
            del chunk, nonskill_chunk
            gc.collect()
        
        # Combine all control chunks
        nonskill_df = pd.concat(nonskill_chunks) if nonskill_chunks else pd.DataFrame(columns=needed_columns)
        
        # Limit control size for efficiency
        control_size = 3 * len(skill_df)
        if len(nonskill_df) > control_size:
            nonskill_df = nonskill_df.sample(control_size, random_state=42)
        
        # Control group count = len(nonskill_df)
        
        # Check if we have a valid control group
        if len(nonskill_df) == 0:
            print(f"Warning: Empty control group for {skill}")
            return None, skill_df, nonskill_df
            
        # Combine datasets
        reg_df = pd.concat([skill_df, nonskill_df])
        
        # Overall count = len(reg_df)
        
        # Convert variables as needed
        reg_df['has_skill'] = reg_df['has_skill'].astype(float)
        reg_df['log_salary'] = np.log(reg_df['SALARY_IMPUTED'].astype(float))
    
        # Additional verification to ensure counts are accurate
        treatment_count = sum(reg_df['has_skill'] == 1)
        control_count = sum(reg_df['has_skill'] == 0)
        
        # Verify treatment count matches original
        if treatment_count != len(skill_df):
            print(f"Warning: Treatment count mismatch for {skill}. Expected {len(skill_df)}, got {treatment_count}")
            
        # Verify control count matches limit (if applicable)
        if len(nonskill_df) > control_size and control_count != control_size:
            print(f"Warning: Control count mismatch for {skill}. Expected {control_size}, got {control_count}")
    
        return reg_df, skill_df, nonskill_df
        
    except Exception as e:
        print(f"Error creating regression dataset for skill {skill}:")
        import traceback
        print(traceback.format_exc())
        return None, None, None

In [16]:
def run_regression_with_fixest(skill, reg_df, skill_df, nonskill_df, exp_as_discrete, soc_code, soc_title):
    """
    Run regression for a specific SOC group using pyfixest with SOC_2021_5 fixed effects
    """
    try:
        # Store key metrics - these are correctly calculating the counts
        n_obs = len(reg_df)  # Overall count
        n_skill = len(skill_df) if skill_df is not None else 0  # Treatment count
        n_control = len(nonskill_df) if nonskill_df is not None else 0  # Control count
        
        # Verification step - make sure counts are consistent
        if reg_df is not None:
            actual_treatment = sum(reg_df['has_skill'] == 1)
            actual_control = sum(reg_df['has_skill'] == 0)
            
            if actual_treatment != n_skill:
                print(f"Warning: Treatment count mismatch in regression. Expected {n_skill}, got {actual_treatment}")
                n_skill = actual_treatment  # Update to actual count
                
            if actual_control != n_control:
                print(f"Warning: Control count mismatch in regression. Expected {n_control}, got {actual_control}")
                n_control = actual_control  # Update to actual count
                
            # Verify total matches
            if actual_treatment + actual_control != n_obs:
                print(f"Warning: Total count mismatch. Treatment + Control = {actual_treatment + actual_control}, Total = {n_obs}")
        
        # Check for collinearity within this specific SOC group
        if soc_code != 'All':  # Only for specific SOC groups
            # Check variation of has_skill within fixed effect categories
            soc5_var = reg_df.groupby('SOC_2021_5')['has_skill'].nunique()
            edu_var = reg_df.groupby('MIN_EDULEVELS_NAME')['has_skill'].nunique()
            
            # If has_skill doesn't vary within fixed effect categories, skip regression
            if soc5_var.max() <= 1 or edu_var.max() <= 1:
                print(f"    Insufficient variation for {skill} in {soc_title}. Skipping regression.")
                return {
                    'term': skill,
                    'soc_code': soc_code,
                    'soc_title': soc_title,
                    'estimate': None,
                    'std.error': None,
                    'p.value': None,
                    'sig': 0,
                    'n_obs': n_obs,
                    'n_skill': n_skill,
                    'n_control': n_control,
                    'exp_as_discrete': exp_as_discrete
                }
        
        # Convert categorical variables to strings for pyfixest
        reg_df['SOC_2021_5'] = reg_df['SOC_2021_5'].astype(str)
        reg_df['YEAR'] = reg_df['YEAR'].astype(str)
        reg_df['MIN_EDULEVELS_NAME'] = reg_df['MIN_EDULEVELS_NAME'].astype(str)
        
        # Prepare fixed effect formula based on whether experience is discrete
        if exp_as_discrete:
            reg_df['MIN_YEARS_EXPERIENCE'] = reg_df['MIN_YEARS_EXPERIENCE'].astype(str)
            formula = "log_salary ~ has_skill | SOC_2021_5 + YEAR + MIN_EDULEVELS_NAME + MIN_YEARS_EXPERIENCE"
        else:
            reg_df['MIN_YEARS_EXPERIENCE'] = reg_df['MIN_YEARS_EXPERIENCE'].astype(float)
            formula = "log_salary ~ has_skill + MIN_YEARS_EXPERIENCE | SOC_2021_5 + YEAR + MIN_EDULEVELS_NAME"
        
        # Try-except block for the regression to catch collinearity errors
        try:
            # Run fixed effects regression using pyfixest
            results = feols(formula, data=reg_df, vcov="HC1")
            
            # Extract results - Using methods
            try:
                # Get coefficients dictionary 
                coef_dict = results.coef()
                estimate = coef_dict.get('has_skill', None)
                
                # Get standard errors
                se_dict = results.se()
                std_error = se_dict.get('has_skill', None)
                
                # Get p-values
                pvalue_dict = results.pvalue()
                p_value = pvalue_dict.get('has_skill', None)
                
                # Calculate dollar premiums
                if estimate is not None:
                    mean_salary = reg_df[reg_df['has_skill'] == 0]['SALARY_IMPUTED'].mean()
                    percentage_premium = (np.exp(estimate) - 1) * 100
                    dollar_premium = (np.exp(estimate) - 1) * mean_salary
                else:
                    mean_salary = None
                    percentage_premium = None
                    dollar_premium = None
                    
            except Exception as method_error:
                print(f"Method access failed: {method_error}")
                estimate, std_error, p_value = None, None, None
                mean_salary, percentage_premium, dollar_premium = None, None, None
                
        except ValueError as ve:
            if "All variables are collinear" in str(ve):
                print(f"    Collinearity detected for {skill} in {soc_title}. Skipping regression.")
                return {
                    'term': skill,
                    'soc_code': soc_code,
                    'soc_title': soc_title,
                    'estimate': None,
                    'std.error': None,
                    'p.value': None,
                    'sig': 0,
                    'n_obs': n_obs,
                    'n_skill': n_skill,
                    'n_control': n_control,
                    'exp_as_discrete': exp_as_discrete
                }
            else:
                # Re-raise other ValueErrors
                raise
        
        # Create result dictionary
        result_dict = {
            'term': skill,
            'soc_code': soc_code,
            'soc_title': soc_title,
            'estimate': estimate,
            'std.error': std_error,
            'p.value': p_value,
            'sig': 1 if p_value is not None and p_value < 0.05 else 0,
            'percentage_premium': percentage_premium,
            'dollar_premium': dollar_premium,
            'mean_salary': mean_salary,
            'n_obs': n_obs,          # Overall count
            'n_skill': n_skill,      # Treatment count
            'n_control': n_control,  # Control count
            'exp_as_discrete': exp_as_discrete
        }
        
        # Clean up
        del results
        gc.collect()
        
        return result_dict
        
    except Exception as e:
        print(f"Error in regression for {skill} and SOC {soc_code}:")
        import traceback
        print(traceback.format_exc())
        
        # Return a structured error result
        return {
            'term': skill,
            'soc_code': soc_code,
            'soc_title': soc_title,
            'estimate': None,
            'std.error': None,
            'p.value': None,
            'sig': 0,
            'percentage_premium': None,
            'dollar_premium': None,
            'mean_salary': None,
            'n_obs': n_obs if 'n_obs' in locals() else 0,
            'n_skill': n_skill if 'n_skill' in locals() else 0,
            'n_control': n_control if 'n_control' in locals() else 0,
            'exp_as_discrete': exp_as_discrete
        }

In [17]:
def wage_premium_finder_by_soc(skill, reg_df, skill_df, nonskill_df, exp_as_discrete=False):
    """
    Calculates wage premium for a given skill across different SOC groups.
    Using SOC_2021_5 for fixed effects in the regression.
    
    Parameters:
    skill (str): The skill to analyze
    reg_df (DataFrame): Combined regression dataset
    skill_df (DataFrame): Dataset of posts with the skill
    nonskill_df (DataFrame): Dataset of posts without the skill
    exp_as_discrete (bool): If True, experience is treated as discrete
    
    Returns:
    list: Results of the wage premium analysis for each SOC group
    """
    try:
        if reg_df is None or len(nonskill_df) == 0:
            result = {
                'term': skill,
                'soc_code': 'All',
                'soc_title': 'All SOCs',
                'estimate': None,
                'std.error': None,
                'p.value': None,
                'sig': 0,
                'n_obs': len(skill_df) if skill_df is not None else 0,
                'n_skill': len(skill_df) if skill_df is not None else 0,
                'n_control': 0,
                'exp_as_discrete': exp_as_discrete
            }
            return [result]
        
        # List to store results for each SOC group
        results = []
        
        # Verify counts in the combined dataset
        total_obs = len(reg_df)
        total_skill = sum(reg_df['has_skill'] == 1)
        total_control = sum(reg_df['has_skill'] == 0)
        
        if total_skill + total_control != total_obs:
            print(f"Warning: Count discrepancy for {skill}. Total: {total_obs}, Skill: {total_skill}, Control: {total_control}")
        
        # Create a result for all SOCs combined first
        all_soc_result = run_regression_with_fixest(
            skill, reg_df, skill_df, nonskill_df, exp_as_discrete, 
            soc_code='All', soc_title='All SOCs'
        )
        results.append(all_soc_result)
        
        # Now run separate regressions for each SOC group
        soc_groups = reg_df['SOC_2021_2'].unique()
        
        for soc_code in soc_groups:
            # Skip if not enough data
            soc_filter = reg_df['SOC_2021_2'] == soc_code
            soc_reg_df = reg_df[soc_filter].copy()
            
            if len(soc_reg_df) < 100:  # Skip if too few observations
                continue
            
            # Create subset dataframes for this SOC
            soc_skill_df = skill_df[skill_df['SOC_2021_2'] == soc_code].copy() if skill_df is not None else None
            soc_nonskill_df = nonskill_df[nonskill_df['SOC_2021_2'] == soc_code].copy() if nonskill_df is not None else None
            
            # Skip if not enough skill observations
            if soc_skill_df is None or len(soc_skill_df) < 30:
                continue
                
            # Verify counts for this SOC group
            soc_total = len(soc_reg_df)
            soc_skill_count = sum(soc_reg_df['has_skill'] == 1)
            soc_control_count = sum(soc_reg_df['has_skill'] == 0)
            
            if soc_skill_count != len(soc_skill_df):
                print(f"Warning: Treatment count mismatch for {skill} in SOC {soc_code}")
            
            if soc_control_count != len(soc_nonskill_df):
                print(f"Warning: Control count mismatch for {skill} in SOC {soc_code}")
            
            # Get SOC title
            soc_title = soc_reg_df['SOC_2021_2_NAME'].iloc[0] if len(soc_reg_df) > 0 else f"SOC {soc_code}"
            
            # Run regression for this SOC group
            soc_result = run_regression_with_fixest(
                skill, soc_reg_df, soc_skill_df, soc_nonskill_df, 
                exp_as_discrete, soc_code, soc_title
            )
            results.append(soc_result)
            
            # Free memory
            del soc_reg_df, soc_skill_df, soc_nonskill_df
            gc.collect()
            
        return results
        
    except Exception as e:
        print(f"Error in wage premium analysis for skill {skill}:")
        import traceback
        print(traceback.format_exc())
        return None

In [18]:
def analyze_by_soc(reg_df, skills, batch_size=5):
    """
    Analyze wage premiums across different SOC groups using SOC_2021_5 fixed effects
    
    Parameters:
    reg_df (DataFrame): The dataset with job postings
    skills (list): List of skills to analyze
    batch_size (int): Number of skills to process per batch
    
    Returns:
    DataFrame: Results of the analysis
    """
    all_results = []
    
    # Process skills in batches
    for i in range(0, len(skills), batch_size):
        batch_skills = skills[i : i + batch_size]
        batch_num = i//batch_size + 1
        print(f"\nProcessing batch {batch_num}, skills {i+1} to {min(i+batch_size, len(skills))}")
        
        for skill in batch_skills:
            print(f"  Processing skill: {skill}")
            
            # Create regression datasets with SOC_2021_5 columns
            reg_dataset, skill_df, nonskill_df = create_regression_dataset(skill, reg_df)
            
            if reg_dataset is not None:
                # Track dataset sizes for verification
                n_total = len(reg_dataset)
                n_treatment = sum(reg_dataset['has_skill'] == 1)
                n_control = sum(reg_dataset['has_skill'] == 0)
                print(f"    Dataset created - Total: {n_total}, Treatment: {n_treatment}, Control: {n_control}")
                
                # Run regression with discrete experience across SOC groups
                results = wage_premium_finder_by_soc(
                    skill, reg_dataset, skill_df, nonskill_df, exp_as_discrete=True
                )
                if results is not None:
                    all_results.extend(results)
                    # Report results for all SOCs
                    all_soc_result = next((r for r in results if r['soc_code'] == 'All'), None)
                    if all_soc_result and all_soc_result['estimate'] is not None:
                        print(f"    Overall result: estimate={all_soc_result['estimate']:.4f}, p={all_soc_result['p.value']:.4f}")
                        print(f"    Counts: n_obs={all_soc_result['n_obs']}, n_skill={all_soc_result['n_skill']}, n_control={all_soc_result['n_control']}")
                    else:
                        print(f"    No valid overall result.")
            else:
                print(f"    Could not create dataset for {skill}")
            
            # Clear memory
            del reg_dataset, skill_df, nonskill_df
            gc.collect()
        
        # Force memory cleanup after batch
        gc.collect()
    
    # Convert to DataFrame
    print(f"Processing final results for SOC-specific analysis...")
    results_df = pd.DataFrame(all_results)
    
    if len(results_df) == 0:
        print("No results found!")
        return pd.DataFrame()
    
    # Calculate additional metrics - handle None values
    results_df['wage_diff_pct'] = results_df.apply(
        lambda row: (np.exp(row['estimate']) - 1) if row['sig'] == 1 and row['estimate'] is not None else 0, 
        axis=1
    )
    
    # Rank within each SOC group
    results_df['rank'] = results_df.groupby('soc_code')['estimate'].rank(method='dense', ascending=False)
    
    # Add skill statistics
    try:
        # Try to handle different index formats
        if isinstance(skill_stats.index, pd.Index):
            stats_df = skill_stats.reset_index()
            if 'SKILL_NAME' in stats_df.columns:
                stats_df = stats_df.rename(columns={'SKILL_NAME': 'term'})
            else:
                stats_df = stats_df.rename(columns={'index': 'term'})
        else:
            stats_df = skill_stats.reset_index().rename(columns={'index': 'term'})
            
        # Only merge relevant stats to save memory
        relevant_stats = stats_df[stats_df['term'].isin(results_df['term'].unique())]
        results_df = results_df.merge(relevant_stats, on='term')
        
        # Free memory
        del stats_df, relevant_stats
        gc.collect()
    except Exception as e:
        print(f"Error adding skill statistics: {e}")
    
    # Add skill categories
    try:
        # Load categories 
        skill_categories = pd.read_sql(
            'SELECT * FROM TEMPORARY_DATA.JNANIA."ADE_LC_skills_v5_2025_03_10"',
            conn
        )
        
        # Only get needed categories
        categories_subset = skill_categories[skill_categories['Skill'].isin(results_df['term'].unique())]
        results_df = results_df.merge(
            categories_subset.rename(columns={'Skill': 'term'}),
            on='term'
        )
        
        # Calculate weighted wage premiums within SOC groups and categories
        results_df['sh_skill_by_soc_category'] = results_df.groupby(['soc_code', 'Category Name'])['obs_count'].transform(
            lambda x: x / x.sum()
        )
        results_df['wage_diff_pct_weighted'] = (
            results_df['wage_diff_pct'] * results_df['sh_skill_by_soc_category']
        )
        
        # Free memory
        del categories_subset, skill_categories
        gc.collect()
    except Exception as e:
        print(f"Could not add skill categories: {e}")
    
    # # Add flag for small sample sizes
    # results_df['small_sample'] = results_df['n_skill'] < 500
    
    return results_df

In [19]:
def process_and_save_results(results_df):
    """
    Process results from SOC_2021_5 fixed effects analysis and save to file
    
    Parameters:
    results_df (DataFrame): The results dataframe to process and save
    
    Returns:
    tuple: (detailed_df, summary_df, filename) - The processed dataframes and output filename
    """
    # Update filename to reflect the SOC_2021_5 fixed effects
    filename = 'ADE_wage_premiums_by_soc_SOC5FE_fixest_blue_collar.csv'
    
    # Add timestamp to filename
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    filename = f'ADE_wage_premiums_by_soc_SOC5FE_fixest_{timestamp}.csv'
    
    # Save full results
    results_df.to_csv(filename, index=False)
    print(f"SOC-specific results saved to {filename}")
    
    # Create summary by SOC and category
    try:
        # Summary with all samples
        summary_df = results_df.groupby(['soc_code', 'soc_title', 'Category Name']).agg({
            'wage_diff_pct_weighted': 'sum',
            'term': 'count',
            'n_obs': 'sum',
            'n_skill': 'sum',
            'n_control': 'sum'
        }).rename(columns={
            'wage_diff_pct_weighted': 'category_wage_premium',
            'term': 'skill_count',
            'n_obs': 'total_postings',
            'n_skill': 'treatment_postings',
            'n_control': 'control_postings'
        }).reset_index()
        
        # Add percent of postings that are in treatment group
        summary_df['treatment_ratio'] = summary_df['treatment_postings'] / summary_df['total_postings']
        
        # Summary filename
        summary_filename = f'ADE_wage_premiums_summary_by_soc_category_SOC5FE_{timestamp}.csv'
        summary_df.to_csv(summary_filename, index=False)
        print(f"Summary by SOC and category saved to {summary_filename}")
        
    except Exception as e:
        print(f"Error creating summary: {e}")
        import traceback
        print(traceback.format_exc())
        summary_df = pd.DataFrame()
    
    return results_df, summary_df, filename

In [20]:
def filter_skills_for_collinearity(reg_df, skills, min_occ_count=1, min_ed_count=1):
    """
    Filter skills to remove those that would be collinear with SOC_2021_5 fixed effects
    """
    # Filter the dataset to only include specified skills
    skill_df = reg_df[reg_df['SKILL_NAME'].isin(skills)]
    
    # Group by skill and calculate counts
    skill_stats = skill_df.groupby('SKILL_NAME').agg({
        'SOC_2021_5': 'nunique',  # Using SOC_2021_5 instead of SOC_2021_2
        'MIN_EDULEVELS_NAME': 'nunique',
        'ID': 'count'
    }).rename(columns={
        'SOC_2021_5': 'soc5_count',  # Renamed to reflect SOC_2021_5
        'MIN_EDULEVELS_NAME': 'ed_count',
        'ID': 'obs_count'
    })
    
    # Filter skills based on occupation and education counts
    valid_skills = skill_stats[
        (skill_stats['soc5_count'] > min_occ_count) & 
        (skill_stats['ed_count'] > min_ed_count)
    ].index.tolist()
    
    # Add stats for skill counts by SOC_2021_2 for later use
    soc2_counts = skill_df.groupby('SKILL_NAME')['SOC_2021_2'].nunique()
    skill_stats['soc2_count'] = soc2_counts
    
    # Make this available globally
    globals()['skill_stats'] = skill_stats
    
    print(f"Filtered skills: {len(valid_skills)}/{len(skills)} remain after filtering")
    print(f"  - Skills must appear in more than {min_occ_count} SOC_2021_5 categories")
    print(f"  - Skills must appear in more than {min_ed_count} education levels")
    
    return valid_skills

In [21]:
def run_soc_analysis_fixest(reg_df, skills, skill_stats, conn=None):
    """
    Run analysis comparing wage premiums across SOC groups using pyfixest with SOC_2021_5 fixed effects.
    
    Parameters:
    reg_df (DataFrame): The dataset with job postings
    skills (list): List of skills to analyze
    skill_stats (DataFrame): Skill statistics
    conn: Database connection
    
    Returns:
    tuple: (detailed_df, summary_df, filename) - The two dataframes and filename
    """
    # First, create a slimmer version of reg_df with only needed columns
    needed_columns = ['ID', 'SALARY_IMPUTED', 'YEAR', 
                      'SOC_2021_5', 'SOC_2021_5_NAME',  # Added SOC_2021_5 fields
                      'SOC_2021_2', 'SOC_2021_2_NAME',
                      'MIN_EDULEVELS_NAME', 'MIN_YEARS_EXPERIENCE', 'SKILL_NAME']
    reg_df_slim = reg_df[needed_columns].copy()
    
    # Filter skills to avoid collinearity issues - JUST LIKE R CODE
    filtered_skills = filter_skills_for_collinearity(reg_df_slim, skills, min_occ_count=1, min_ed_count=1)
    
    # Make skill_stats available globally
    globals()['skill_stats'] = skill_stats
    
    # Run SOC-specific analysis
    print("\n=== RUNNING SOC-SPECIFIC ANALYSIS WITH SOC_2021_5 FIXED EFFECTS USING PYFIXEST ===")
    results_df = analyze_by_soc(reg_df_slim, filtered_skills, batch_size=5)
    detailed_df, summary_df, filename = process_and_save_results(results_df)
    print(f"Analysis complete. Results saved to {filename}")
    
    # Generate summary statistics for each SOC group
    soc_summary = detailed_df[detailed_df['soc_code'] != 'All'].groupby('soc_code').agg({
        'soc_title': 'first',
        'term': 'nunique',
        'n_obs': 'sum',
        'n_skill': 'sum',  # Added treatment count
        'n_control': 'sum',  # Added control count
        'wage_diff_pct': ['mean', 'median', 'min', 'max']
    })
    
    print("\nSummary by SOC group:")
    print(soc_summary)
    
    # Also generate summary by skill category
    try:
        category_summary = detailed_df[detailed_df['soc_code'] == 'All'].groupby('Category Name').agg({
            'term': 'count',
            'wage_diff_pct': ['mean', 'median', 'min', 'max']
        }).sort_values(('wage_diff_pct', 'median'), ascending=False)
        
        print("\nSummary by skill category (all SOCs):")
        print(category_summary)
    except Exception as e:
        print(f"Could not generate category summary: {e}")
    
    return detailed_df, summary_df, filename

In [22]:
# Check for salary-related columns
salary_cols = [col for col in reg_df.columns if 'salary' in col.lower()]
print(f"Salary columns found: {salary_cols}")

Salary columns found: ['SALARY_IMPUTED']


In [23]:
Dont run regression 

SyntaxError: invalid syntax (2039198965.py, line 1)

In [ ]:
# Main execution example
if __name__ == "__main__":
    print("Starting wage premium analysis by SOC code using SOC_2021_5 fixed effects with pyfixest...")
    
    # Run the analysis with SOC_2021_5 fixed effects
    detailed_df, summary_df, filename = run_soc_analysis_fixest(reg_df, include_skills, skill_stats, conn)
    
    print(f"\nAnalysis complete. Results saved to {filename}")
    print(f"The two dataframes are available as 'detailed_df' and 'summary_df'")

In [ ]:
# This is to run additional analysis without having to re-run the regressions
detailed_df = pd.read_csv('ADE_wage_premiums_by_soc_SOC5FE_fixest_20250803_2219.csv')

In [ ]:
detailed_df

In [ ]:
detailed_df['wage_diff_pct_floor'] = np.maximum(detailed_df['wage_diff_pct'], 0)

In [ ]:
soc5_by_soc2 = reg_df.groupby('SOC_2021_2')['SOC_2021_5'].nunique()
print("SOC_2021_5 categories per SOC_2021_2 group:")
print(soc5_by_soc2)

In [ ]:
# Incude counts SOC_2021_5 categories 
regression_soc5_counts = []

# For the full dataset
all_soc5_count = reg_df['SOC_2021_5'].nunique()
regression_soc5_counts.append({
    'soc_code': 'All', 
    'soc_title': 'All SOCs',
    'soc5_count_used': all_soc5_count
})

# For each SOC_2021_2 group
for soc_code in reg_df['SOC_2021_2'].unique():
    soc_df = reg_df[reg_df['SOC_2021_2'] == soc_code]
    soc_title = soc_df['SOC_2021_2_NAME'].iloc[0]
    soc5_count = soc_df['SOC_2021_5'].nunique()
    
    regression_soc5_counts.append({
        'soc_code': soc_code,
        'soc_title': soc_title,
        'soc5_count_used': soc5_count
    })

# Convert to dataframe and merge
soc5_counts_df = pd.DataFrame(regression_soc5_counts)
detailed_df = detailed_df.merge(soc5_counts_df, on=['soc_code', 'soc_title'], how='left')

In [ ]:
detailed_df

In [ ]:
# Create a timestamp for the filename
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# Export to CSV with a descriptive filename
output_filename = f'ADE_wage_premiums_by_soc_SOC5FE_with_soc5_counts_{timestamp}.csv'
detailed_df.to_csv(output_filename, index=False)

print(f"Results with SOC_2021_5 counts exported to {output_filename}")

In [ ]:
# Get the top 10 skills with highest wage premium across all SOCs
top_skills_overall = detailed_df[detailed_df['soc_code'] == 'All'].nlargest(10, 'wage_diff_pct')
print(top_skills_overall[['term', 'wage_diff_pct']])

In [ ]:
# First, let's fix the data types
data_to_clean = {
    'term': 'string',
    'soc_code': 'string',
    'soc_title': 'string',
    'estimate': 'float',
    'std.error': 'float',
    'p.value': 'float',
    'sig': 'int',
    'n_obs': 'int',
    'n_skill': 'int',
    'n_control': 'int',
    'exp_as_discrete': 'bool',
    'wage_diff_pct': 'float',
    'wage_diff_pct_floor': 'float',
    'rank': 'float',  # Keep as float since it might contain decimal rankings
    'soc5_count': 'int',  # Updated from soc_count to soc5_count
    'soc2_count': 'int',  # Added for SOC_2021_2 counts
    'ed_count': 'int',
    'obs_count': 'int',
    'dollar_premium': 'float'  # Added for dollar premium
    # 'small_sample': 'bool'  # Added for our small sample flag
}

# Convert the columns to appropriate types
for col, dtype in data_to_clean.items():
    if col in detailed_df.columns:
        if dtype == 'float':
            detailed_df[col] = pd.to_numeric(detailed_df[col], errors='coerce')
        elif dtype == 'int':
            detailed_df[col] = pd.to_numeric(detailed_df[col], errors='coerce').astype('Int64')
        elif dtype == 'bool':
            if detailed_df[col].dtype == 'object':
                detailed_df[col] = detailed_df[col].map({'True': True, 'False': False})

# Check if we already have category data
if 'Category Name' not in detailed_df.columns:
    # Query skill categories
    category_query = """
    SELECT "Skill", "Category Name" 
    FROM TEMPORARY_DATA.JNANIA."ADE_LC_skills_v5_2025_03_10"
    """
    skill_categories = pd.read_sql(category_query, conn)
    
    # Merge with your detailed dataframe
    df_with_categories = detailed_df.merge(
        skill_categories,
        left_on='term',
        right_on='Skill',
        how='left'
    )
    
    # Keep both term and Skill columns
    df_with_categories = df_with_categories.drop('Skill', axis=1)
    
    # Check for any skills that didn't match
    missing_categories = df_with_categories[df_with_categories['Category Name'].isna()]['term'].unique()
    if len(missing_categories) > 0:
        print(f"Warning: {len(missing_categories)} skills did not match with categories:")
        print(missing_categories)
else:
    # Categories already added
    df_with_categories = detailed_df

# Generate timestamp for filenames
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# Save the results with categories
output_file = f'ADE_wage_premiums_by_soc_SOC5FE_with_categories_{timestamp}.csv'
df_with_categories.to_csv(output_file, index=False)
print(f"Saved detailed results with categories to {output_file}")

# Create a summary by SOC and category
summary_by_category = df_with_categories.groupby(['soc_code', 'soc_title', 'Category Name']).agg({
    'wage_diff_pct': ['mean', 'median'],
    'dollar_premium': ['mean', 'median'],  # Added dollar premium aggregation
    'term': 'nunique',
    'n_obs': 'sum',
    'n_skill': 'sum',
    'n_control': 'sum'
    # 'small_sample': 'mean'  # Percentage of skills with small samples
}).reset_index()

# Flatten the MultiIndex columns
summary_by_category.columns = ['_'.join(col).strip('_') for col in summary_by_category.columns.values]

# Rename columns for clarity
summary_by_category = summary_by_category.rename(columns={
    'wage_diff_pct_mean': 'avg_wage_premium_pct',
    'wage_diff_pct_median': 'median_wage_premium_pct',
    'dollar_premium_mean': 'avg_dollar_premium',
    'dollar_premium_median': 'median_dollar_premium',
    'term_nunique': 'skill_count',
    'n_obs_sum': 'total_observations',
    'n_skill_sum': 'treatment_observations',
    'n_control_sum': 'control_observations',
    # 'small_sample_mean': 'pct_small_samples'
})

# Add treatment ratio
summary_by_category['treatment_ratio'] = summary_by_category['treatment_observations'] / summary_by_category['total_observations']

# Save the category summary
summary_file = f'ADE_wage_premiums_summary_by_category_SOC5FE_{timestamp}.csv'
summary_by_category.to_csv(summary_file, index=False)
print(f"Saved category summary to {summary_file}")

In [ ]:
summary_by_category

In [ ]:
df_with_categories

In [ ]:
# Filter for statistically significant results (p-value < 0.05)
significant_results = df_with_categories[df_with_categories['p.value'] < 0.05]
# # Also filter out small samples if desired
# significant_reliable_results = significant_results[~significant_results['small_sample']]

# Create a timestamp for filenames
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# Create a summary by SOC and category with only significant results
summary_by_category_sig = significant_results.groupby(['soc_code', 'soc_title', 'Category Name']).agg({
    'wage_diff_pct': ['mean', 'median', 'min', 'max'],
    'dollar_premium': ['mean', 'median', 'min', 'max'],  # Added dollar premium
    'mean_salary': ['mean', 'median', 'min', 'max'],     # Added mean salary
    'term': 'nunique',
    'n_obs': 'sum',
    'n_skill': 'sum',
    'n_control': 'sum',
    # 'small_sample': 'mean'  # Percentage of skills with small samples
}).reset_index()

# Flatten the MultiIndex columns
summary_by_category_sig.columns = ['_'.join(col).strip('_') for col in summary_by_category_sig.columns.values]

In [ ]:
summary_by_category_sig

In [ ]:
# Rename columns for clarity
summary_by_category_sig = summary_by_category_sig.rename(columns={
    'wage_diff_pct_mean': 'avg_wage_premium_pct',
    'wage_diff_pct_median': 'median_wage_premium_pct',
    'wage_diff_pct_min': 'min_wage_premium_pct',
    'wage_diff_pct_max': 'max_wage_premium_pct',
    'dollar_premium_mean': 'avg_dollar_premium',
    'dollar_premium_median': 'median_dollar_premium',
    'dollar_premium_min': 'min_dollar_premium',
    'dollar_premium_max': 'max_dollar_premium',
    'term_nunique': 'skill_count',
    'n_obs_sum': 'total_observations',
    'n_skill_sum': 'treatment_observations',
    'n_control_sum': 'control_observations',
    # 'small_sample_mean': 'pct_small_samples'
})

# Add treatment ratio
summary_by_category_sig['treatment_ratio'] = summary_by_category_sig['treatment_observations'] / summary_by_category_sig['total_observations']

# Add a new rank column within each SOC title, based on avg_wage_premium_pct (descending order)
summary_by_category_sig['category_rank_within_soc'] = summary_by_category_sig.groupby(['soc_code', 'soc_title'])['avg_wage_premium_pct'].rank(ascending=False)

# Save the category summary with only significant results
sig_summary_file = f'ADE_wage_premiums_significant_summary_by_category_SOC5FE_{timestamp}.csv'
summary_by_category_sig.to_csv(sig_summary_file, index=False)
print(f"Saved significant results category summary to {sig_summary_file}")

In [ ]:
summary_by_category_sig

In [ ]:
df = summary_by_category_sig

# Display "ALL SOCs" seperately
# Get unique SOC titles including "All SOCs" for faceting
soc_titles = np.append(["All SOCs"], df[df['soc_title'] != 'All SOCs']['soc_title'].unique())

# Define a custom color palette for categories
categories = df['Category Name'].unique()
category_colors = {
    'Adaptive & Reflective Rationality': '#3498db',                  # Blue
    'Quantitative & Statistical Reasoning': '#2ecc71',               # Green
    'Structured & Systematic Decision-Making Process': '#e74c3c'     # Red
}

# Set up the main plot with more vertical space per facet
fig, axes = plt.subplots(len(soc_titles), 1, figsize=(14, 5 * len(soc_titles)), sharex=True)

# Create more active title - now positioned closer to the plot
active_title = "Quantitative & Statistical Reasoning Skills Drive Higher Wage Premiums Across Occupations"
fig.suptitle(active_title, fontsize=20, y=1.01, fontweight='bold')

# Add a subtitle for context - positioned closer to the plot
subtitle = "Analysis of statistically significant wage premiums by occupation and skill category"
plt.figtext(0.5, 0.975, subtitle, ha='center', fontsize=14, style='italic')

# Create the legend at the top - positioned between title and subtitle
legend_handles = [plt.Rectangle((0,0), 1, 1, color=category_colors[cat]) for cat in categories]
fig.legend(legend_handles, categories, 
           loc='upper center', 
           bbox_to_anchor=(0.5, 1.04),  # Position above the plot but below the title
           ncol=len(categories), 
           fontsize=12, 
           frameon=True)

# Loop through each SOC title to create facets
for i, soc_title in enumerate(soc_titles):
    # Filter data for this SOC
    soc_data = df[df['soc_title'] == soc_title].copy()
    
    # Sort by wage premium (descending) for better visualization
    soc_data = soc_data.sort_values('avg_wage_premium_pct', ascending=False)  # Updated column name
    
    # Get the correct axis (handle both single and multiple subplots)
    ax = axes[i] if len(soc_titles) > 1 else axes
    
    # Set up y positions with more space between bars
    y_positions = np.arange(len(soc_data)) * 2  # Double the spacing between bars
    
    # Plot the bars
    for j, (_, row) in enumerate(soc_data.iterrows()):
        category = row['Category Name']
        wage_premium = row['avg_wage_premium_pct']  # Updated column name
        bar_color = category_colors[category]
        
        # Plot bar with increased height
        bar = ax.barh(y_positions[j], wage_premium, height=0.8, color=bar_color, alpha=0.8)
        
        # Position for annotations depends on whether wage premium is positive or negative
        text_x_position = max(wage_premium + 0.01, 0.01) if wage_premium >= 0 else min(wage_premium - 0.01, -0.01)
        text_alignment = 'left' if wage_premium >= 0 else 'right'
        
        # Add category name with better positioning - moved further left
        ax.text(-0.40, y_positions[j], f"{category}", va='center', ha='left', fontsize=11, 
                fontweight='bold', color='black')
        
        # Add wage premium value
        ax.text(text_x_position, y_positions[j], f"{wage_premium:.3f}", va='center', ha=text_alignment, 
                fontsize=11, color='black', fontweight='bold')
        
        # Add rank, skill count and observations as a small annotation with better positioning - moved further left
        annotation = f"Rank: {int(row['category_rank_within_soc'])} | Skills: {int(row['skill_count'])} | Obs: {int(row['total_observations']):,}"
        ax.text(-0.40, y_positions[j]-0.4, annotation, va='center', ha='left', fontsize=9, 
                color='dimgray')
    
    # Make the "All SOCs" panel stand out with a different background if it's the "All SOCs" panel
    if soc_title == "All SOCs":
        ax.set_facecolor('#f8f9fa')  # Light gray background
        ax.set_title(f"Across All Occupations (All SOCs)", fontsize=16, pad=10, fontweight='bold')
    else:
        ax.set_title(f"{soc_title}", fontsize=14, pad=10)
    
    # Add zero line
    ax.axvline(x=0, color='black', linestyle='-', alpha=0.5, linewidth=0.8)
    
    # Remove y-ticks since we have labels
    ax.set_yticks([])
    
    # Set y-axis limits to ensure proper spacing
    ax.set_ylim(min(y_positions)-1.5, max(y_positions)+1.5)
    
    # Add grid lines for easier reading
    ax.grid(axis='x', linestyle='--', alpha=0.3)
    
    # Add x-label only to the bottom subplot
    if i == len(soc_titles) - 1:
        ax.set_xlabel('Wage Premium (%)', fontsize=12)
    
    # Extend x-axis to make more room for labels on the left
    max_abs_value = df['avg_wage_premium_pct'].abs().max()  # Updated column name
    buffer = max_abs_value * 0.3  # Add 30% buffer
    ax.set_xlim(-0.45, max_abs_value+buffer)  # Extended left side further for labels

# Key adjustments to reduce space between subtitle and charts:
plt.tight_layout(rect=[0, 0.02, 1, 0.93])  # Increased top from 0.95 to 0.93 to bring charts up
plt.subplots_adjust(hspace=0.4, top=0.93)  # Reduced from 0.6 to 0.4 for less space between subplots

# Save figure
plt.savefig('wage_premium_by_occupation.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Ensure output directory exists
output_dir = 'occupation_charts'
os.makedirs(output_dir, exist_ok=True)

# Get unique SOC titles including "All SOCs" for separate charts
soc_titles = np.append(["All SOCs"], df[df['soc_title'] != 'All SOCs']['soc_title'].unique())

# Define a custom color palette for categories
categories = df['Category Name'].unique()
category_colors = {
    'Adaptive & Reflective Rationality': '#3498db',                  # Blue
    'Quantitative & Statistical Reasoning': '#2ecc71',               # Green
    'Structured & Systematic Decision-Making Process': '#e74c3c'     # Red
}

# Loop through each SOC title to create separate charts
for soc_title in soc_titles:
    # Filter data for this SOC
    soc_data = df[df['soc_title'] == soc_title].copy()
    
    # Check if we have data for this SOC
    if len(soc_data) == 0:
        print(f"No data found for {soc_title}, skipping...")
        continue
    
    print(f"Creating chart for {soc_title} with {len(soc_data)} data points")
    
    # Sort by wage premium (descending) for better visualization
    soc_data = soc_data.sort_values('avg_wage_premium_pct', ascending=False)  # Updated column name
    
    # Create new figure for each SOC - using plt.figure() to ensure a fresh figure
    plt.close('all')  # Close any existing figures
    fig = plt.figure(figsize=(14, 6))
    ax = fig.add_subplot(111)
    
    # Set up y positions with more space between bars
    y_positions = np.arange(len(soc_data)) * 2  # Double the spacing between bars
    
    # Print debug info
    print(f"Y positions: {y_positions}")
    
    # Plot the bars
    for j, (_, row) in enumerate(soc_data.iterrows()):
        category = row['Category Name']
        # The wage premium is already in percentage format from your updated column
        wage_premium = row['avg_wage_premium_pct']  # Updated column name
        
        print(f"  - Category: {category}, Premium: {wage_premium}%")
        
        # Floor negative values at 0 for display
        display_wage_premium = max(0, wage_premium)
        
        bar_color = category_colors[category]
        
        # Plot bar with increased height
        bar = ax.barh(y_positions[j], display_wage_premium, height=0.8, color=bar_color, alpha=0.8)
        
        # Position for annotations depends on whether wage premium is positive or negative
        text_x_position = max(display_wage_premium + 0.01, 0.01)
        
        # Add category name with better positioning - moved further left
        ax.text(-40, y_positions[j], f"{category}", va='center', ha='left', fontsize=11, 
                fontweight='bold', color='black')
        
        # Add wage premium value or N/A for negative values
        if wage_premium < 0:
            ax.text(text_x_position, y_positions[j], "N/A", va='center', ha='left', 
                    fontsize=11, color='black', fontweight='bold')
        else:
            ax.text(text_x_position, y_positions[j], f"{wage_premium:.1f}%", va='center', ha='left', 
                    fontsize=11, color='black', fontweight='bold')
        
        # Add rank, skill count and observations as a small annotation with better positioning
        annotation = f"Rank: {int(row['category_rank_within_soc'])} | Skills: {int(row['skill_count'])} | Obs: {int(row['total_observations']):,}"
        ax.text(-40, y_positions[j]-0.4, annotation, va='center', ha='left', fontsize=9, 
                color='dimgray')
    
    # Set background style for "All SOCs" panel
    if soc_title == "All SOCs":
        ax.set_facecolor('#f8f9fa')  # Light gray background
        chart_title = "Across All Occupations (All SOCs)"
    else:
        chart_title = soc_title
    
    # Add title and subtitle
    plt.suptitle(chart_title, fontsize=16, y=0.98, fontweight='bold')
    plt.title("Wage Premiums by Skill Category", fontsize=14, pad=10)
    
    # Add zero line
    ax.axvline(x=0, color='black', linestyle='-', alpha=0.5, linewidth=0.8)
    
    # Remove y-ticks since we have labels
    ax.set_yticks([])
    
    # Set y-axis limits to ensure proper spacing
    if len(y_positions) > 0:
        ax.set_ylim(min(y_positions)-1.5, max(y_positions)+1.5)
    
    # Add grid lines for easier reading
    ax.grid(axis='x', linestyle='--', alpha=0.3)
    
    # Add x-label
    ax.set_xlabel('Wage Premium (%)', fontsize=12)
    
    # Create legend at the top
    legend_handles = [plt.Rectangle((0,0), 1, 1, color=category_colors[cat]) for cat in categories]
    ax.legend(legend_handles, categories, 
              loc='upper right',
              fontsize=10, 
              frameon=True)
    
    # Extend x-axis to make more room for labels on the left
    max_premium = max(df['avg_wage_premium_pct'].max(), 5)  # Updated column name, no need to multiply by 100
    buffer = max_premium * 0.3  # Add 30% buffer
    ax.set_xlim(-45, max_premium+buffer)  # Extended left side further for labels
    
    # Adjust layout
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    # Create filename-safe version of SOC title
    safe_filename = soc_title.replace('/', '_').replace(' ', '_').replace(',', '').replace('&', 'and')
    filename = f'{output_dir}/{safe_filename}_wage_premium.png'
    
    # Ensure the figure is drawn before saving
    fig.canvas.draw()
    
    # Save figure with debugging
    print(f"Saving figure to {filename}")
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    
    # Show figure
    plt.show()
    
    # Close the figure explicitly
    plt.close(fig)

print(f"All charts saved to {output_dir}/ directory")

In [ ]:
# Filter for only statistically significant results (p-value < 0.05)
sig_df = detailed_df[detailed_df['p.value'] < 0.05].copy()

# Step 1: Pivot the data to get skills as rows and SOC codes as columns
# First, exclude the "All SOCs" category to focus on specific occupations
soc_specific_df = sig_df[sig_df['soc_code'] != 'All']

# Create the pivot table with wage_diff_pct as values
pivot_df = soc_specific_df.pivot_table(
    index='term',
    columns='soc_code',
    values='wage_diff_pct',
    aggfunc='first'  # Use first since we should have one value per skill-SOC combination
)

# Step 2: Calculate the range of wage premiums for each skill across SOCs
# This measures how much the premium varies between occupations
pivot_df['premium_range'] = pivot_df.max(axis=1) - pivot_df.min(axis=1)

# Step 3: Identify skills that have both positive and negative premiums
# Create indicators for positive and negative premiums
for col in pivot_df.columns:
    if col != 'premium_range':
        pivot_df[f'{col}_is_positive'] = (pivot_df[col] > 0)
        pivot_df[f'{col}_is_negative'] = (pivot_df[col] < 0)

# Count how many positive and negative premiums each skill has
positive_cols = [col for col in pivot_df.columns if col.endswith('_is_positive')]
negative_cols = [col for col in pivot_df.columns if col.endswith('_is_negative')]

pivot_df['positive_count'] = pivot_df[positive_cols].sum(axis=1)
pivot_df['negative_count'] = pivot_df[negative_cols].sum(axis=1)

# Filter for skills that have both positive and negative premiums
contrasting_skills = pivot_df[(pivot_df['positive_count'] > 0) & (pivot_df['negative_count'] > 0)]

# Step 4: Sort by the range to find skills with the largest differences
contrasting_skills = contrasting_skills.sort_values('premium_range', ascending=False)

# Drop the indicator columns for cleaner output
cols_to_drop = positive_cols + negative_cols + ['positive_count', 'negative_count']
contrasting_skills = contrasting_skills.drop(columns=cols_to_drop)

# Step 5: Get the top skills with contrasting effects
top_contrasting_skills = contrasting_skills.head(20)

# To see which SOCs have positive vs negative for each skill, create a summary
skill_contrast_summary = []

for skill in top_contrasting_skills.index:
    skill_data = pivot_df.loc[skill].drop(['premium_range', 'positive_count', 'negative_count'] + 
                                          positive_cols + negative_cols)
    
    # Find the SOC code with the highest positive premium
    max_positive_soc = skill_data[skill_data > 0].idxmax() if any(skill_data > 0) else None
    min_negative_soc = skill_data[skill_data < 0].idxmin() if any(skill_data < 0) else None
    
    # Get the SOC title for the highest positive premium
    if max_positive_soc:
        max_positive_title = soc_specific_df[soc_specific_df['soc_code'] == max_positive_soc]['soc_title'].iloc[0]
    else:
        max_positive_title = ""
    
    # Get the SOC title for the most negative premium
    if min_negative_soc:
        min_negative_title = soc_specific_df[soc_specific_df['soc_code'] == min_negative_soc]['soc_title'].iloc[0]
    else:
        min_negative_title = ""
    
    # Get the highest positive and most negative values
    max_positive = skill_data[skill_data > 0].max() if any(skill_data > 0) else 0
    min_negative = skill_data[skill_data < 0].min() if any(skill_data < 0) else 0
    
    skill_contrast_summary.append({
        'Skill': skill,
        'Premium Range': contrasting_skills.loc[skill, 'premium_range'],
        'Highest Positive': max_positive,
        'Most Negative': min_negative,
        'Positive SOCs': max_positive_title,
        'Negative SOCs': min_negative_title
    })

# Create a summary dataframe
contrast_summary_df = pd.DataFrame(skill_contrast_summary)

# Save the results
contrast_summary_df.to_csv('skills_with_contrasting_premiums.csv', index=False)
top_contrasting_skills.to_csv('top_contrasting_skills_detailed.csv')

print(f"Found {len(contrasting_skills)} skills with both positive and negative wage premiums across occupations.")
print(f"Top skill with contrasting premiums: {contrast_summary_df.iloc[0]['Skill']} with a premium range of {contrast_summary_df.iloc[0]['Premium Range']:.4f}")

# Display the summary table
contrast_summary_df

In [ ]:
def create_chart_from_summary(summary_df, n_skills=5):
    """
    Create a diverging bar chart from the summary dataframe with fixed SOC title handling
    
    Parameters:
    -----------
    summary_df : pandas DataFrame
        Your contrast_summary_df with Skill, Highest Positive, Most Negative, etc.
    n_skills : int
        Number of top skills to display
    """
    import textwrap
    
    # Get top N skills
    top_skills = summary_df.head(n_skills)
    
    # Create a figure with a subplot for each skill
    fig, axes = plt.subplots(n_skills, 1, figsize=(18, 3*n_skills), sharex=True)
    
    # Ensure axes is always a list even with a single skill
    if n_skills == 1:
        axes = [axes]
    
    # For each skill, create a simplified horizontal bar chart showing highest positive and most negative
    for i, (_, row) in enumerate(top_skills.iterrows()):
        ax = axes[i]
        skill = row['Skill']
        
        # Get highest positive occupation
        pos_val = row['Highest Positive']
        pos_soc = row['Positive SOCs']  # Keep the entire string
        
        # Get most negative occupation
        neg_val = row['Most Negative']
        neg_soc = row['Negative SOCs']  # Keep the entire string
        
        # Create a dictionary of values
        values = {
            'Highest Positive\n' + pos_soc: pos_val,
            'Most Negative\n' + neg_soc: neg_val
        }
        
        # Sort by value
        sorted_items = sorted(values.items(), key=lambda x: x[1])
        labels = [item[0] for item in sorted_items]
        vals = [item[1] for item in sorted_items]
        
        # Set up positions for the bars
        y_pos = np.arange(len(labels))
        
        # Create bars with different colors
        bars = ax.barh(y_pos, vals, height=0.6)
        bars[0].set_color('#d62728')  # Red for negative
        bars[1].set_color('#2ca02c')  # Green for positive
        
        # Remove the y-ticks completely
        ax.set_yticks([])
        
        # Custom positioning for the labels - moved further left
        for j, label in enumerate(labels):
            # Split label into two parts: title and SOC
            parts = label.split('\n')
            if len(parts) == 2:
                title, soc = parts
                # First line (title) in bold - moved much further left
                ax.text(-1.5, y_pos[j], title, ha='left', va='bottom', fontsize=10, fontweight='bold')
                # Second line (SOC) with smaller font, wrapped to multiple lines if needed
                wrapped_soc = '\n'.join(textwrap.wrap(soc, width=40))  # Wrap text at 40 characters
                ax.text(-1.5, y_pos[j]-0.15, wrapped_soc, ha='left', va='top', fontsize=9)
            else:
                # Fallback if there's no newline
                ax.text(-1.5, y_pos[j], label, ha='left', va='center', fontsize=10)
        
        # Add values inside/near the bars with better positioning
        for j, v in enumerate(vals):
            if v < 0:
                # For negative values, position inside the bar near the left edge
                x_pos = v * 0.5  # Position at middle of bar
                ha = 'center'
                color = 'white'  # White text for contrast against red bar
            else:
                # For positive values, position to the right of the bar
                x_pos = v + 0.02
                ha = 'left'
                color = 'black'
            
            ax.text(x_pos, y_pos[j], f'{v:.3f}', va='center', ha=ha, fontsize=10, 
                    fontweight='bold', color=color)
        
        # Set title and labels for this subplot
        ax.set_title(f'{skill}', fontsize=14, pad=10)
        
        # Add a premium range annotation
        range_text = f"Premium Range: {row['Premium Range']:.3f}"
        ax.annotate(range_text, xy=(0.98, 0.05), xycoords='axes fraction', 
                    ha='right', fontsize=10, bbox=dict(boxstyle="round,pad=0.3", 
                                                      fc='#f0f0f0', ec="gray", alpha=0.7))
        
        # Add gridlines
        ax.grid(axis='x', linestyle='--', alpha=0.6)
        
        # Add a vertical line at x=0
        ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
        
        # Set x axis limits to ensure bars are clearly visible and leave room for labels on the left
        max_abs = max(abs(min(vals)), abs(max(vals)))
        buffer = max_abs * 0.3
        ax.set_xlim(-1.6, max_abs+buffer)  # Extended leftward to make room for labels
    
    # Add a common x-label for all subplots
    fig.text(0.5, 0.02, 'Wage Premium', ha='center', fontsize=14)
    
    # Add a title for the entire figure
    fig.suptitle('Skills with Most Contrasting Wage Premiums Across Occupations', 
                 fontsize=16, y=1.02, fontweight='bold')
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.subplots_adjust(hspace=0.5)
    
    # Save the figure
    plt.savefig('contrasting_skills_summary_bar.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
create_chart_from_summary(contrast_summary_df, n_skills=5)

In [ ]:
df_with_categories

In [ ]:
# Ensure output directory exists
output_dir = 'occupation_charts'
os.makedirs(output_dir, exist_ok=True)

# Set style for better-looking charts
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

# For Jupyter notebooks, use this magic command
%matplotlib inline

# First, create a filtered dataset of significant results
significant_reliable = df_with_categories[
    (detailed_df['p.value'] < 0.05) 
]

In [ ]:
significant_reliable

In [ ]:
# Chart 1: Weighted average wage premium by category
# First, filter for significant results without small samples
# AND exclude "All SOCs" to avoid double counting
significant_reliable_by_soc = significant_reliable[
    significant_reliable['soc_code'] != 'All'  # Exclude the "All SOCs" results
]

# Calculate the weighted average by category
# Using n_skill as weights
category_weighted = significant_reliable_by_soc.groupby('Category Name').apply(
    lambda x: np.average(x['wage_diff_pct'], weights=x['n_skill'])
).reset_index()
category_weighted.columns = ['Category Name', 'weighted_wage_premium']

# Create a percentage version of the weighted_wage_premium for plotting
category_weighted['weighted_wage_premium_pct'] = category_weighted['weighted_wage_premium'] * 100

# Sort by premium descending
category_weighted = category_weighted.sort_values('weighted_wage_premium', ascending=False)

# Create the chart
plt.figure(figsize=(12, 8))
barplot = sns.barplot(
    x='weighted_wage_premium_pct',  # Use the percentage column for the x-axis
    y='Category Name', 
    data=category_weighted,
    palette='viridis'
)

# Add value labels with percentages
for i, v in enumerate(category_weighted['weighted_wage_premium_pct']):
    barplot.text(v + 0.2, i, f"{v:.1f}%", va='center')  # Adjusted position and format

plt.title('Weighted Average Wage Premium by Skill Category', fontsize=16)
plt.xlabel('Wage Premium (%)', fontsize=14)
plt.ylabel('Skill Category', fontsize=14)

# Add a note about weighting methodology
plt.figtext(0.5, 0.02, "Note: Wage premiums are weighted by the number of skills in each category", 
            ha='center', fontsize=12, fontstyle='italic')

plt.tight_layout(rect=[0, 0.03, 1, 0.97])  # Adjust layout to make room for the note

# Save the chart first, then display it
plt.savefig('occupation_charts/wage_premium_by_category_weighted.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close('all')  # Close all figures

In [ ]:
# Ensure output directory exists
output_dir = 'occupation_charts'
os.makedirs(output_dir, exist_ok=True)

# Set style for better-looking charts
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

# For Jupyter notebooks, use this magic command
%matplotlib inline

# First, create a filtered dataset of significant results 
significant_reliable = detailed_df[
    (detailed_df['p.value'] < 0.05) 
]

# Filter for significant results in "All SOCs" only
all_socs_sig = significant_reliable[significant_reliable['soc_code'] == 'All']

# Sort by wage premium
all_socs_sig_sorted = all_socs_sig.sort_values('wage_diff_pct', ascending=False)

# Get top 5, middle 5, and bottom 5
top_5 = all_socs_sig_sorted.head(5)
middle_5 = all_socs_sig_sorted.iloc[len(all_socs_sig_sorted)//2-2:len(all_socs_sig_sorted)//2+3]
bottom_5 = all_socs_sig_sorted.tail(5).copy()  # Create a copy to avoid warning

# Modify: Floor negative wage premiums at 0 for bottom skills while preserving the original order
# Create a display column for plotting that floors negative values at 0
bottom_5['wage_diff_pct_display'] = bottom_5['wage_diff_pct'].clip(lower=0)

# Combine with a group label
top_5['group'] = 'Top 5'
middle_5['group'] = 'Middle 5'
bottom_5['group'] = 'Bottom 5'

# Create display column for top and middle as well (same as original value)
top_5['wage_diff_pct_display'] = top_5['wage_diff_pct']
middle_5['wage_diff_pct_display'] = middle_5['wage_diff_pct']

# Combine the dataframes
combined = pd.concat([top_5, middle_5, bottom_5])

# Create the chart using the display column for x-axis
plt.figure(figsize=(12, 10))

# Convert wage premium to percentage for display (multiply by 100)
for df in [top_5, middle_5, bottom_5]:
    df['wage_diff_pct_display_pct'] = df['wage_diff_pct_display'] * 100

# Combine the dataframes
combined = pd.concat([top_5, middle_5, bottom_5])

# Plot all bars first (this will ensure all y-labels are shown)
barplot = sns.barplot(
    x='wage_diff_pct_display_pct',  # Use percentage values
    y='term', 
    data=combined,
    hue='group',
    palette={
        'Top 5': 'forestgreen',
        'Middle 5': 'gold',
        'Bottom 5': 'lightgray'  # Light gray for bottom 5
    },
    dodge=False
)

# Add hatching to the bottom 5 bars
bottom_hatch = '///'
for i, row in enumerate(combined.itertuples()):
    if row.group == 'Bottom 5':
        # Get the bar and add hatching
        bar = barplot.patches[i]
        bar.set_hatch(bottom_hatch)
        bar.set_edgecolor('firebrick')
        bar.set_linewidth(1.5)

# Add value labels (single set of labels to avoid duplicates)
for i, row in enumerate(combined.itertuples()):
    if row.group == 'Bottom 5':
        # Show "N/A" for bottom 5
        barplot.text(0.1, i, "N/A", va='center', fontweight='bold')
    else:
        # Show the original value formatted as percentage
        # Convert decimal to percentage for the label
        pct_value = row.wage_diff_pct * 100
        barplot.text(pct_value + 0.2, i, f"{pct_value:.1f}%", va='center')

plt.title('Top, Middle, and Bottom 5 Skills by Wage Premium', fontsize=16)
plt.xlabel('Wage Premium (%)', fontsize=14)
plt.ylabel('Skill', fontsize=14)
plt.axvline(x=0, color='black', linestyle='-', alpha=0.7)
plt.legend(title='Group')

# Add a note about N/A skills
plt.figtext(0.5, 0.02, "Note: N/A skills represent those with no wage premium", 
            ha='center', fontsize=12, fontstyle='italic')

plt.tight_layout(rect=[0, 0.03, 1, 0.97])  # Adjust layout to make room for the note

# Save the chart first, then display it
plt.savefig('occupation_charts/top_middle_bottom_skills_all.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close('all')  # Close all figures

print("Chart created successfully and saved as:")
print("- occupation_charts/top_middle_bottom_skills_all.png")


In [ ]:
# Get the top 10 skills with highest wage premium across all SOCs
top_skills_overall = detailed_df[detailed_df['soc_code'] == 'All'].nlargest(10, 'wage_diff_pct')
print(top_skills_overall[['term', 'wage_diff_pct']])

In [ ]:
## Test the sampling error (results 1 and 2 should be the same if we uncomment out the 500,000 sample)

In [ ]:
# Run 1
results1 = analyze_by_soc(reg_df, ["Adaptive Reasoning"], batch_size=1)



Processing batch 1, skills 1 to 1
  Processing skill: Adaptive Reasoning


In [ ]:
reg_df_new = pd.read_sql(combined_query, conn)

In [ ]:
# Run 2 immediately after  
results2 = analyze_by_soc(reg_df_new, ["Adaptive Reasoning"], batch_size=1)